<a href="https://colab.research.google.com/github/asrjy/d2l-notes/blob/master/Chapter%207%20-%20Convolutional%20Neural%20Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks

Flattening image layers means losing the spatial relationship between the pixels. In order to overcome this issue, we use CNNs instead of regular MLPs

Since MLPs don't care about the spatial relationship, we get the same results even though the order of the pixels are modified. This phenomenon is not wanted. 

Modern CNNs tend to be computationally effieicent, require fewer parameters than fully connected networks and easy to parallelize over multiple GPU cores. CNNs have also shown decent performance on one dimenstionals data such as audio, time series data where recurrent neural networks are conventionally used. 

## From Fully Connected Layers to Convolutions

For low dimensional data where we lack the knowledge to construct specific architectures that identify patterns that seek interactions among features, an MLP may be the best we can do. However, for high dimensional perceptual data, such structure less networks can gro unwieldy. 

Convolutional Neural Networks are one way to exploit the structure in natural images. 

### Invariance

Spatial Invariance allows CNNs to learn useful representations with fewer parameters. What it basically means is our model should identify a pig in air or a plane in water. Where the target is located is not of importance, it's existence is what we look for. 

There are two different types of invariances. 

1 - In the earliest layers, our network should respond similarly to the same patch, regardless of where the patch is located in the image. A patch here means a part of the image on which the NN works on. This principle is called translation invariance / translation equivariance. 

2 - The earliest layers of the network should focus on local regions, without regard for the contents in distant regions. This is called locality principle. Eventually they are aggregated to make predictions on the image as a whole. 

3 - As we proceed, deeper layers should capture longer range features of the image similar to higher level vision in nature. 

### Constraining the MLP

Assume the image is represented as X with shape (i, j) and the hidden representation is represented as H with shape also (i, j) where each element of the hidden representation is calculated by summing over the pixels of X centered at i, j weighted by V (i, j, a, b)

\begin{split}\begin{aligned} \left[\mathbf{H}\right]_{i, j} &= [\mathbf{U}]_{i, j} + \sum_k \sum_l[\mathsf{W}]_{i, j, k, l}  [\mathbf{X}]_{k, l}\\ &=  [\mathbf{U}]_{i, j} +
\sum_a \sum_b [\mathsf{V}]_{i, j, a, b}  [\mathbf{X}]_{i+a, j+b}.\end{aligned}\end{split}

The indices a, b run over both positive and negative offsets ocvering the entire image. 

#### Translation Invariance

According to this principle, a shift in X should lead to a shift in the hidden representation. This is only possible if V and U do not depepnd on i, j. As a result we represent V(i, j, a, b) as just V((a, b) and U as a constant. 

Now the simplified representation is 

$[\mathbf{H}]_{i, j} = u + \sum_a\sum_b [\mathbf{V}]_{a, b}  [\mathbf{X}]_{i+a, j+b}$.

The above concept is called convolution where we are effectively weighing pixels at (i+a j+b) in the vicinity of (i, j) with coefficients V(a, b) to obtain the hidden representation. 

V(a, b) requires far fewer parameters than V(i, j, a, b) since it no longer depends on i and j. 

#### Locality

According to this principle, we should not look very far from X(i, j) to get relevant information about what is going on at X(i, j). This means, after some value of a and b, V(a, b) need to be 0. 

Now the hidden representation becomes, 

$[\mathbf{H}]_{i, j} = u + \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} [\mathbf{V}]_{a, b}  [\mathbf{X}]_{i+a, j+b}$.

The value of delta is typically smaller than 10. 

The above equation is called a convolutional layer. V is referred to as Convolution kernel/filter. 

Without this layer, for a single megeapixel image, we would require billions of parameters, but with the convolutional layer, we would require a few hundred, without altering the dimensionality of either the inputs or the hidden representations. 

The cost of this reduction of in parameters is that, we only capture local information while determining the value of each hidden activation. 

This bias might not always agree with reality, as there could be images that are not translation invariant. 

### Convolutions

In mathematics, a convolution operation between two functions is the measure of overlap between f and g when g is flipped and shifted by x. 

$(f * g)(\mathbf{x}) = \int f(\mathbf{z}) g(\mathbf{x}-\mathbf{z}) d\mathbf{z}.$

When we are dealing with discrete objects, the integral in the beginning, becomes a sum. 

$(f * g)(i) = \sum_a f(a) g(i-a).$

For two dimensional tensors, we have corresponding indices a and b for i and j. 

$(f * g)(i, j) = \sum_a\sum_b f(a, b) g(i-a, j-b).$

This is similar to the convolution operation we arrived at before excluding the + instead of -. The more proper name for the equation we got before is cross-correlation. 

### Channels

To support multiple channels in both the inputs and the hidden representations, we add two more coordinates to V and one more to X. 

$[\mathsf{H}]_{i,j,d} = \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} \sum_c [\mathsf{V}]_{a, b, c, d} [\mathsf{X}]_{i+a, j+b, c},$

where d indexes the output channels in the hidden representation. 

### Exercises 

1 - Assume that the size of the convolution kernel is $\Delta = 0$. Show that in this case the convolution kernel implements an MLP independently for each set of channels. This leads to the Network in Network architectures [Lin et al., 2013].

Ans - When the size of convolution layer is 0, it means, nearby data of each coordinate is not considered. There's no convolution happening. The shape of V and X are going to be the same. This is what happens in a regular MLP. 

2.1 - Audio data is often represented as a one-dimensional sequence. When might you want to impose locality and translation invariance for audio?

Ans - Inclusion of locality and translation invariance means, audio at one part is not necessarily related to audio at another part. This wouldn't work for speech related problems, but would work for pattern recognition problems where local part doesnt have any relation to parts outside its locality.

3 - Why might translation invariance not be a good idea after all? Give an example.

Ans - Not a good idea in cases where one patch of the image has a relationship with it's position. An example could be when identifying the face of a human, if the position of eyes and mouth are exchanged, the prediction probably would not be a human. 

4 - Do you think that convolutional layers might also be applicable for text data? Which problems might you encounter with language?

Ans - Convolutional Layers include translational invariance and Locality. Translation Invariance would not work on text data as it is sequential data. Locality would not be of much help since words make sense only when combined with other words. 

5 - What happens with convolutions when an object is at the boundary of an image.

Ans - Values around the border are decided using padding. 

## Convolutions for Images

### The Cross-Correlation Operation

If the input is of size n_h \times n_w and the kernel is of size $k_h \times k_w$, the output after the cross correlation operation would be of size $(n_h-k_h+1) \times (n_w-k_w+1).$

This is because we need enough space to shift the kernel across the image. With padding, the output size will not be varied as we pad zeros around the boundary so there is enough space to shift the kernel. 

In [1]:
!pip install d2l

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 82 kB 732 kB/s 
     |████████████████████████████████| 9.9 MB 11.4 MB/s 
     |████████████████████████████████| 15.7 MB 8.9 MB/s 
     |████████████████████████████████| 11.2 MB 16.0 MB/s 
     |████████████████████████████████| 61 kB 9.5 MB/s 
     |████████████████████████████████| 944 kB 36.9 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    U

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
  h, w = K.shape
  Y = torch.zeros((X.shape[0] - h + 1) , (X.shape[1] - w + 1))
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i, j] = (X[i: i+h, j:j+w] * K).sum()
  return Y 

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

### Convolutional Layers

A convolutional layer performs cross correlation  between input and kernel and adds a scalar bias to it to produce am output. The two parameters of a convolutional layer are the kernel and the bias. When training, we typically intialize them randomly. 

Implementing a convolutional layer based on the corr2d function defined above. 

In [5]:
class Conv2D(nn.Module):
  def __init__(self, kernel_size):
    super().__init__()
    self.weight = nn.Parameter(torch.rand(kernel_size))
    self.bias = nn.Parameter(torch.zeros(1))
  def forward(self, X):
    return corr2d(X, self.weight) + self.bias

### Object Edge Detection in Images

Detecting the edge of an object can be performed by finding the location of change of color in piels. 

In [6]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

When we perform cross correlation with a kernel of size 1x2, if the two elements match with the elements in the patch of the image, it outputs 0 ie., at location i, j it calculates x(i, j) - x(i+1, j)

In [7]:
K = torch.tensor([[1.0, -1.0]])

In [8]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [9]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

It does not work because, the cross correlation can only detect vertical edge, not horizontal edges. 

### Learning a Kernel

Although the above process is nice, it's hard to define each kernel when we are working with bigger architectures. We want to kernel to automatically learn these processes. 

First we construct a convolution layer and initialize it randomly. In each iteration we will use the squared error to compare Y with the output of the convolution layer, calculate the gradient to update the kernel. 

In [10]:
# Convolutional Layer with 1 output channel and kernel fo shape (1, 2). Ignoring the bias for now. 
conv2d = nn.LazyConv2d(1, kernel_size = (1, 2), bias = False)

# (Example, Channel, Height, Width)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(10):
  Y_hat = conv2d(X)
  l = (Y_hat - Y) ** 2
  conv2d.zero_grad()
  l.sum().backward()
  # Updating the kernel
  conv2d.weight.data[:] -= lr * conv2d.weight.grad
  if (i+1)%2 == 0:
    print(f"Epoch {i+1}, loss {l.sum():.3f}")

Epoch 2, loss 11.266
Epoch 4, loss 2.136
Epoch 6, loss 0.459
Epoch 8, loss 0.118
Epoch 10, loss 0.037


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [11]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0008, -0.9659]])

### Cross-Correlation and Convolution

In order to perform just convolution instead of cross correlation, we need to flip the kernel both horizontally and vertically, then perform cross correlation with the input tensor. 

Since kernels are learnt from data, it doesnt matter whether layers perform cross correlation or convolution. The output remains the same. 

Meaning if a layer performs cross correlation and it's weights are represented as K, the learned kernel be K', K' will be the same even when K is flipped horizontally and vertically. 

### Feature Map and Receptive Field

The outpt of Convolution Layer is sometimes called Feature Map. Receptive field of any element x of any layer means the elements from the previous layers, that cal affect the calculation of x during the forward propagation. It may be larger than the actual size of the input. 

## Padding and Stride 

10 Convolution Layers of kernel size 5x5 on a 240x240 image, reduce the output size to 200x200. Padding and Strided Convolutions offer more control over the size of the output. 

### Padding

One straightforward solution to this issue is to add zeros around the image. If we add ph rows of padding (half on top and the rest on bottom) and pw columns of padding (half on left and the rest on right), the output shape would be 

(nh - kh + ph + 1) x (nw - kw + pw + 1). 

In many cases ph = kh - 1 and pw = kw - 1 to give input and output the same height and width. 

CNNs commonly use conv kernels of odd height and width. This means when we pad, we can divide the number of rows and columns by 2, thus having equal row and column paddings on both sides. 


In [12]:
import torch
from torch import nn

def comp_conv2d(conv2d, X):
  # conv2d requires images of 4 dimensions. Adding the example and channel dimension
  X = X.reshape((1, 1) + X.shape)
  Y = conv2d(X)
  # stripping the example and channel dimensions
  return Y.reshape(Y.shape[2:])

# 1 row and column are padded on either side, so total of 2 rows or columns are added
conv2d = nn.LazyConv2d(1, kernel_size = 3, padding = 1)

X = torch.rand(size = (8, 8))
comp_conv2d(conv2d, X).shape

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([8, 8])

In [14]:
# Using a convolution layer with different height and width but selecting different padding to get the same output shape

conv2d = nn.LazyConv2d(1, kernel_size = (5, 3), padding = (2, 1))
comp_conv2d(conv2d, X).shape

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([8, 8])

### Stride 

For computational efficiency, we might want to slide the conv layer more than 1 step at a time, skipping the intermediate locations. 

Number of rows and columns per traversed per slide is called stride. SO far we used strides 1. 

If the stride height is sh, stride width is sw, the output shape is 

$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$

If we set ph = kh - 1 and pw = kw - 1, the output shape is simplified to 

$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$. 

In [15]:
conv2d = nn.LazyConv2d(1, kernel_size = 3, padding = 1, stride = 2)
comp_conv2d(conv2d, X).shape

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([4, 4])

In [17]:
conv2d = nn.LazyConv2d(1, kernel_size = (3, 5), padding = (0, 1), stride = (3, 4))
comp_conv2d(conv2d, X).shape

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 2])

## Multiple Input and Multiple Output Channels

Notion of channels is as old as CNNs themselves (Since 1995 almost)

### Multiple Input Channels

The convolution kernel must have the same number of channels as the input layer. The outputs of the kernel at each channel are then added that result in a single channel kernel output. 


In [18]:
import torch
from d2l import torch as d2l

def corr2d_multi_channel(X, K):
  # Iterate through each dimension of K (each channel of K) then add them
  return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [19]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_channel(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

### Multiple Output Channels

In the above example, no matter the number nof inputs, we end up getting a single dimension output. But it is essential to have multiple channel outputs. In most popular neural networks we add more channels to the output as we go in depth into the neural network architecture. This is done to compensate for the lack of spatial resolution. Intuitively, this can be thought of as each channel corresponding to a different set of features. 

If ci and co are the number of input and output channels, kh and kw are kernel height and width, to get multi channel kernel output we create kernel of shape ci x kh x kw for each output channel co. We concatenate them on the output channel dimension so that shape is co x ci x kh x kw. 

In [20]:
def corr2d_multi_channel_kernel_output(X, K):
  # Iterate through the 0th dimension of 'K' and each time perform 
  # cross correlation operations with input 'X'. All of the results are stacked
  # together
  return torch.stack([corr2d_multi_channel(X, k) for k in K], 0)


In [21]:
K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [22]:
corr2d_multi_channel_kernel_output(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### 1x1 Convolutional Layer

